In [3]:
import psycopg2 as psypg

In [4]:
try:
    conn = psypg.connect(
        host="db-bdt.crc01ueoqr2u.eu-west-1.rds.amazonaws.com",
        database="postgres",
        user="bdtuser",
        password="BDTdatabase2022")

    cur = conn.cursor()

    sql = """select region, sum(population)
            from municipalities
            group by region"""
    cur.execute(sql)
    # commit the changes to the database
    populations = cur.fetchall()
    sql = """SELECT region, count(*)
            FROM places as p 
            join municipalities as m
            on p.municipality = m.istat
            group by region"""
    cur.execute(sql)
    # commit the changes to the database
    places = cur.fetchall()
    # close communication with the database
    cur.close()
except (Exception, psypg.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [5]:
populations

[("Valle d'Aosta", Decimal('127844.0')),
 ('Marche', Decimal('1525945.0')),
 ('Sicilia', Decimal('4999932.0')),
 ('Trentino-Alto Adige', Decimal('1037264.0')),
 ('Campania', Decimal('5769750.0')),
 ('Puglia', Decimal('4050803.0')),
 ('Basilicata', Decimal('576194.0')),
 ('Abruzzo', Decimal('1312507.0')),
 ('Friuli-Venezia Giulia', Decimal('1221901.0')),
 ('Piemonte', Decimal('4379512.0')),
 ('Toscana', Decimal('3676351.0')),
 ('Lazio', Decimal('5557276.0')),
 ('Sardegna', Decimal('1640379.0')),
 ('Molise', Decimal('313341.0')),
 ('Umbria', Decimal('886239.0')),
 ('Veneto', Decimal('4881593.0')),
 ('Liguria', Decimal('1565127.0')),
 ('Calabria', Decimal('1947938.0')),
 ('Emilia-Romagna', Decimal('4367138.0')),
 ('Lombardia', Decimal('9783720.0'))]

In [6]:
places

[('Marche', 990),
 ('Sicilia', 5782),
 ('Trentino-Alto Adige', 2836),
 ('Campania', 1753),
 ('Puglia', 9265),
 ('Basilicata', 180),
 ('Abruzzo', 923),
 ('Friuli-Venezia Giulia', 653),
 ('Piemonte', 6533),
 ('Toscana', 8012),
 ('Lazio', 14155),
 ('Sardegna', 1631),
 ('Molise', 205),
 ('Umbria', 1841),
 ('Veneto', 4990),
 ('Liguria', 2710),
 ('Calabria', 681),
 ('Emilia-Romagna', 2095),
 ('Lombardia', 11250)]

In [8]:
mixed_list = []
for region, pop in populations:
    for pl_region, num_places in places:
        if region == pl_region:
            mixed_list.append((region, pop, num_places))
mixed_list

[('Marche', Decimal('1525945.0'), 990),
 ('Sicilia', Decimal('4999932.0'), 5782),
 ('Trentino-Alto Adige', Decimal('1037264.0'), 2836),
 ('Campania', Decimal('5769750.0'), 1753),
 ('Puglia', Decimal('4050803.0'), 9265),
 ('Basilicata', Decimal('576194.0'), 180),
 ('Abruzzo', Decimal('1312507.0'), 923),
 ('Friuli-Venezia Giulia', Decimal('1221901.0'), 653),
 ('Piemonte', Decimal('4379512.0'), 6533),
 ('Toscana', Decimal('3676351.0'), 8012),
 ('Lazio', Decimal('5557276.0'), 14155),
 ('Sardegna', Decimal('1640379.0'), 1631),
 ('Molise', Decimal('313341.0'), 205),
 ('Umbria', Decimal('886239.0'), 1841),
 ('Veneto', Decimal('4881593.0'), 4990),
 ('Liguria', Decimal('1565127.0'), 2710),
 ('Calabria', Decimal('1947938.0'), 681),
 ('Emilia-Romagna', Decimal('4367138.0'), 2095),
 ('Lombardia', Decimal('9783720.0'), 11250)]

In [15]:
print("Number of facilities per 100.000 people")
for region, pop, pl in mixed_list:
    rate = pl/float(pop)*100000
    print(f"{region} => {round(rate)}")

Number of facilities per 100.000 people
Marche => 65
Sicilia => 116
Trentino-Alto Adige => 273
Campania => 30
Puglia => 229
Basilicata => 31
Abruzzo => 70
Friuli-Venezia Giulia => 53
Piemonte => 149
Toscana => 218
Lazio => 255
Sardegna => 99
Molise => 65
Umbria => 208
Veneto => 102
Liguria => 173
Calabria => 35
Emilia-Romagna => 48
Lombardia => 115


In [18]:
import osmnx as ox
from time import time



leisure_tags = {
    'leisure':[
        'stadium',
    ]
}
address = "Torino, Piemonte, Italy"
start_time = time()
print(f"Doing leisures for {address}...")
leisures = ox.geometries_from_place(address, tags=leisure_tags)

print(f"Leisures done in {time()-start_time}s!")
print(f"Leisures results => {leisures.shape}")

Doing leisures for Torino, Piemonte, Italy...
Leisures done in 2.3980400562286377s!
Leisures results => (7, 18)


In [19]:
leisures

nodes  \
element_type osmid                                                          
way          22954072   [247316024, 7620565081, 247316025, 247316026, ...   
             43138628   [540419580, 540419581, 540419589, 540419590, 5...   
             43708853   [553094165, 553094162, 553094163, 553094164, 1...   
             172821575  [1836725413, 6208291577, 6208291578, 620829157...   
             285499599  [2891765977, 2891765138, 2891762277, 289176554...   
             481253195  [5043258233, 5043235903, 5043235899, 504323589...   
             652725229  [433047738, 433872146, 436555930, 433047736, 4...   

                        leisure  \
element_type osmid                
way          22954072   stadium   
             43138628   stadium   
             43708853   stadium   
             172821575  stadium   
             285499599  stadium   
             481253195  stadium   
             652725229  stadium   

                                                                 geometry  \
element_type osmid                                                          
way          22954072   POLYGON ((7.71824 45.06781, 7.71906 45.06776, ...   
             43138628   POLYGON ((7.63245 45.05818, 7.63263 45.05816, ...   
             43708853   POLYGON ((7.63655 45.03251, 7.63763 45.03214, ...   
             172821575  POLYGON ((7.63996 45.10985, 7.64012 45.10947, ...   
             285499599  POLYGON ((7.63243 45.05932, 7.63267 45.05897, ...   
             481253195  POLYGON ((7.65822 45.03890, 7.65941 45.03850, ...   
             652725229  POLYGON ((7.64915 45.04253, 7.64929 45.04266, ...   

                           alt_name building  \
element_type osmid                             
way          22954072           NaN      NaN   
             43138628   PalaRuffini  stadium   
             43708853           NaN      yes   
             172821575          NaN      NaN   
             285499599          NaN      NaN   
             481253195          NaN      NaN   
             652725229          NaN      NaN   

                                                      name  \
element_type osmid                                           
way          22954072                                  NaN   
             43138628   Palazzetto dello Sport Gianni Asti   
             43708853                 Palaghiaccio Tazzoli   
             172821575                     Allianz Stadium   
             285499599                Stadio Primo Nebiolo   
             481253195                   Stadio Filadelfia   
             652725229       Stadio Olimpico Grande Torino   

                                        name:en   sport start_date  wikidata  \
element_type osmid                                                             
way          22954072                       NaN     NaN        NaN       NaN   
             43138628   PalaRuffini Sport Arena   multi       1961  Q3889165   
             43708853                       NaN     NaN        NaN  Q3889383   
             172821575          Allianz Stadium  soccer       2011    Q62149   
             285499599                      NaN   multi       1959  Q3967860   
             481253195                      NaN  soccer        NaN   Q786449   
             652725229                      NaN     NaN        NaN       NaN   

                                               wikipedia wheelchair  \
element_type osmid                                                    
way          22954072                                NaN        NaN   
             43138628                     it:PalaRuffini        NaN   
             43708853               it:Palasport Tazzoli        yes   
             172821575       it:Allianz Stadium (Torino)        yes   
             285499599  it:Stadio Primo Nebiolo (Torino)        NaN   
             481253195              it:Stadio Filadelfia        NaN   
             652725229                               NaN        NaN   

           